In [1]:
from load_model_and_data import loader, known_skew
from bootstrapper import data_iterator, data_iterator_2d
import numpy as np
from os.path import join
import itertools

In [2]:
# Group Initialization times into overlapping hour periods (e.g., 2100-2200, 2200-2300, )
times_iterator = ['2000', '2030', '2100', '2130', '2200',
       '2230', '2300', '2330','0000', '0030', '0100', '0130', '0200', '0230', '0300',]
group = []
for i in range(len(times_iterator)-2):
    if i%2==0:
        group.append(list(times_iterator[i+d] for d in range(3)))

xdata_init_times=[]
for g in group:
    first_time = g[0][:2]+':'+g[0][2:]
    second_time = g[-1][:2]+':'+g[-1][2:]
    xdata_init_times.append(f'{first_time} - {second_time}')

In [3]:
lead_times = ['first_hour', 'second_hour']
targets = ['tornado', 'severe_hail', 'severe_wind']
n_bootstrap=500

for lead_time, target in itertools.product(lead_times, targets):

    (predictions,
     X, y, 
     init_times, 
     test_dates,
    fti) = loader(lead_time, target)

    X['Run Date'] = test_dates
    # Stratify by initialization times 
    datasets = data_iterator(to_be_stratified=init_times, 
                             iterator=group,
                             predictions=predictions,
                             X=X,
                             y=y,
                             known_skew=known_skew[lead_time][target],
                             n_bootstrap=n_bootstrap,
                             forecast_time_indices = fti,
                            )

    for ds in datasets:
        ds['xdata'] = xdata_init_times
  
    # Save the results
    out_path = '/work/mflora/ML_DATA/stratified_results'
    for ds, mode in zip(datasets, ['ml', 'ml_reduced', 'bl']):
        ds.to_netcdf(join(out_path, f'{mode}_init_stratify_{lead_time}_{target}.nc'))
        ds.close()             

/home/monte.flora/miniconda3/envs/wofs_test/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator SimpleImputer from version 0.24.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/monte.flora/miniconda3/envs/wofs_test/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator StandardScaler from version 0.24.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/monte.flora/miniconda3/envs/wofs_test/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LogisticRegression from vers

/home/monte.flora/miniconda3/envs/wofs_test/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator SimpleImputer from version 0.24.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/monte.flora/miniconda3/envs/wofs_test/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator StandardScaler from version 0.24.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/monte.flora/miniconda3/envs/wofs_test/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LogisticRegression from vers